## Vector Search with IRIS SQL
This tutorial covers how to use IRIS as a vector database. 

For this tutorial, we will use a dataset of 2.2k online reviews of scotch (
dataset from https://www.kaggle.com/datasets/koki25ando/22000-scotch-whisky-reviews) . With our latest vector database functionality, we can leverage the latest embedding models to run semantic search on the online reviews of scotch whiskeys. In addition, we'll be able to apply filters on columns with structured data. For example, we will be able to search for whiskeys that are priced under $100, and are 'earthy, smooth, and easy to drink'. Let's find our perfect whiskey!

In [1]:
import os, pandas as pd
from sentence_transformers import SentenceTransformer
from sqlalchemy import create_engine, text

c:\Users\Usuario\Documents\Projectes\Hack24\hackupc-2024\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
username = 'demo'
password = 'demo'
hostname = os.getenv('IRIS_HOSTNAME', 'localhost')
port = '1972' 
namespace = 'USER'
CONNECTION_STRING = f"iris://{username}:{password}@{hostname}:{port}/{namespace}"

In [3]:
engine = create_engine(CONNECTION_STRING)

In [4]:
# Load the CSV file
df = pd.read_csv('../data/scotch_review.csv')

In [5]:
df.head()

,Unnamed: 0,name,category,review.point,price,currency,description
0,1,"Johnnie Walker Blue Label, 40%",Blended Scotch Whisky,97,225,$,"Magnificently powerful and intense. Caramels, ..."
1,2,"Black Bowmore, 1964 vintage, 42 year old, 40.5%",Single Malt Scotch,97,4500.00,$,What impresses me most is how this whisky evol...
2,3,"Bowmore 46 year old (distilled 1964), 42.9%",Single Malt Scotch,97,13500.00,$,There have been some legendary Bowmores from t...
3,4,"Compass Box The General, 53.4%",Blended Malt Scotch Whisky,96,325,$,With a name inspired by a 1926 Buster Keaton m...
4,5,"Chivas Regal Ultis, 40%",Blended Malt Scotch Whisky,96,160,$,"Captivating, enticing, and wonderfully charmin..."


In [6]:
# Clean data
# Remove the specified columns
df.drop(['currency'], axis=1, inplace=True)

# Drop the first column
df.drop(columns=df.columns[0], inplace=True)

# Remove rows without a price
df.dropna(subset=['price'], inplace=True)

# Ensure values in 'price' are numbers
df = df[pd.to_numeric(df['price'], errors='coerce').notna()]

# Replace NaN values in other columns with an empty string
df.fillna('', inplace=True)

In [7]:
df.head()

,name,category,review.point,price,description
0,"Johnnie Walker Blue Label, 40%",Blended Scotch Whisky,97,225,"Magnificently powerful and intense. Caramels, ..."
1,"Black Bowmore, 1964 vintage, 42 year old, 40.5%",Single Malt Scotch,97,4500.00,What impresses me most is how this whisky evol...
2,"Bowmore 46 year old (distilled 1964), 42.9%",Single Malt Scotch,97,13500.00,There have been some legendary Bowmores from t...
3,"Compass Box The General, 53.4%",Blended Malt Scotch Whisky,96,325,With a name inspired by a 1926 Buster Keaton m...
4,"Chivas Regal Ultis, 40%",Blended Malt Scotch Whisky,96,160,"Captivating, enticing, and wonderfully charmin..."


Now, InterSystems IRIS supports vectors as a datatype in tables! Here, we create a table with a few different columns. The last column, 'description_vector', will be used to store vectors that are generated by passing the 'description' of a review through an embedding model.

In [8]:

with engine.connect() as conn:
    with conn.begin():# Load 
        sql = f"""
                CREATE TABLE scotch_reviews (
        name VARCHAR(255),
        category VARCHAR(255),
        review_point INT,
        price DOUBLE,
        description VARCHAR(2000),
        description_vector VECTOR(DOUBLE, 384)
        )
                """
        result = conn.execute(text(sql))

In [9]:
# Load a pre-trained sentence transformer model. This model's output vectors are of size 384
model = SentenceTransformer('all-MiniLM-L6-v2') 

modules.json: 100%|██████████| 349/349 [00:00<?, ?B/s] 
c:\Users\Usuario\Documents\Projectes\Hack24\hackupc-2024\.venv\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Usuario\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
confi

In [10]:

# Generate embeddings for all descriptions at once. Batch processing makes it faster
embeddings = model.encode(df['description'].tolist(), normalize_embeddings=True)

# Add the embeddings to the DataFrame
df['description_vector'] = embeddings.tolist()


In [11]:
df.head()

,name,category,review.point,price,description,description_vector
0,"Johnnie Walker Blue Label, 40%",Blended Scotch Whisky,97,225,"Magnificently powerful and intense. Caramels, ...","[-0.010494349524378777, 0.014728995971381664, ..."
1,"Black Bowmore, 1964 vintage, 42 year old, 40.5%",Single Malt Scotch,97,4500.00,What impresses me most is how this whisky evol...,"[0.02318122237920761, -0.05123031884431839, 0...."
2,"Bowmore 46 year old (distilled 1964), 42.9%",Single Malt Scotch,97,13500.00,There have been some legendary Bowmores from t...,"[0.04333316907286644, -0.017066601663827896, -..."
3,"Compass Box The General, 53.4%",Blended Malt Scotch Whisky,96,325,With a name inspired by a 1926 Buster Keaton m...,"[-0.0759400948882103, -0.0367622971534729, 0.0..."
4,"Chivas Regal Ultis, 40%",Blended Malt Scotch Whisky,96,160,"Captivating, enticing, and wonderfully charmin...","[-0.012818830087780952, -0.09769783914089203, ..."


In [12]:
with engine.connect() as conn:
    with conn.begin():
        for index, row in df.iterrows():
            sql = text("""
                INSERT INTO scotch_reviews 
                (name, category, review_point, price, description, description_vector) 
                VALUES (:name, :category, :review_point, :price, :description, TO_VECTOR(:description_vector))
            """)
            conn.execute(sql, {
                'name': row['name'], 
                'category': row['category'], 
                'review_point': row['review.point'], 
                'price': row['price'], 
                'description': row['description'], 
                'description_vector': str(row['description_vector'])
            })


Let's look for a scotch that costs less than $100, and has an earthy and creamy taste.

In [21]:
description_search = "acid, charged, good with coca-cola"
search_vector = model.encode(description_search, normalize_embeddings=True).tolist() # Convert search phrase into a vector

In [22]:
with engine.connect() as conn:
    with conn.begin():
        sql = text("""
            SELECT TOP 3 * FROM scotch_reviews 
            WHERE price < 60 
            ORDER BY VECTOR_DOT_PRODUCT(description_vector, TO_VECTOR(:search_vector)) DESC
        """)

        results = conn.execute(sql, {'search_vector': str(search_vector)}).fetchall()


In [23]:
print(results)

[('Johnnie Walker Red Label, 40%', 'Blended Scotch Whisky', 84, 25.0, 'How does this long-established blend shape up on its own without a ginger or soda mixer? Fresh apple juice, a twist of lemon, dried orange peel, pepp ... (122 characters truncated) ... ered spices, with cracked black pepper and a little sourness to finish. Mild, agreeably smooth, without any bombast or hullabaloo; it just blends in.', '-.10379201173782348632,-.12053799629211425781,.021331196650862693786,.065074943006038665771,-.036439534276723861694,.031347528100013732911,-.05065988 ... (8824 characters truncated) ... 3425903,.0077716400846838951111,.037494491785764694213,-.0039177411235868930816,-.036219995468854904174,.052950583398342132568,.033144917339086532592'), (' Dewar’s White Label, 40%', 'Blended Scotch Whisky', 85, 25.0, 'Honey, fudge, a little melon and white grape, dry roasted spices, and peppercorn, but noticeably less fruit than the age-statement varieties. The pal ... (127 characters truncated) ... p

In [24]:
results_df = pd.DataFrame(results, columns=df.columns).iloc[:, :-1] # Remove vector
pd.set_option('display.max_colwidth', None)  # Easier to read description
results_df.head()

,name,category,review.point,price,description
0,"Johnnie Walker Red Label, 40%",Blended Scotch Whisky,84,25.0,"How does this long-established blend shape up on its own without a ginger or soda mixer? Fresh apple juice, a twist of lemon, dried orange peel, peppercorn, cracker bread, and a vegetal spicy note. It gets into its stride with delicious apple, gentle fudge, and good-tempered spices, with cracked black pepper and a little sourness to finish. Mild, agreeably smooth, without any bombast or hullabaloo; it just blends in."
1,"Dewar’s White Label, 40%",Blended Scotch Whisky,85,25.0,"Honey, fudge, a little melon and white grape, dry roasted spices, and peppercorn, but noticeably less fruit than the age-statement varieties. The palate has pulpy white melon, lemon, lime, and some grapefruit acidity, before sweetening with honey. Quite tart at times, some peppery spice collaborates before the vanilla cream and green fruits take charge. A clean, juicy finish of lemon zest and mild spices coats the throat."
2,"The Antiquary, 40%",Blended Scotch Whisky,80,20.0,"This venerable blend has a nose of mandarin syrup cake, vanilla cream, and cut hay, with chopped cilantro and other herbal top notes. Sweet and fruity, though verging on tasting of artificial sweeteners at times, there are malt notes and a developing butteriness, with sweet mandarin notes and sponge cake. It all hangs on a light structure but it’s less smooth on the finish, with a scratch of wood smoke."
